In [2]:
!dir

 Volume in drive C is Acer
 Volume Serial Number is 1497-4303

 Directory of C:\Users\drobi\Desktop\uni\CS4240\DCRNN

22/03/2023  22:02    <DIR>          .
22/03/2023  22:02    <DIR>          ..
22/03/2023  00:58             1,273 .gitignore
22/03/2023  22:02    <DIR>          .ipynb_checkpoints
22/03/2023  00:58    <DIR>          data
22/03/2023  22:02                72 data.ipynb
22/03/2023  00:58             1,273 dcrnn_train.py
22/03/2023  01:12             1,568 experimenting.ipynb
22/03/2023  00:58    <DIR>          figures
22/03/2023  01:01    <DIR>          lib
22/03/2023  00:58             1,088 LICENSE
22/03/2023  01:01    <DIR>          model
22/03/2023  00:58             7,713 README.md
22/03/2023  00:58                84 requirements.txt
22/03/2023  00:58             1,464 run_demo.py
22/03/2023  00:58    <DIR>          scripts
               8 File(s)         14,535 bytes
               8 Dir(s)  64,804,827,136 bytes free


In [12]:
import pandas as pd
import numpy as np

In [20]:
sensor_loc = pd.read_csv('data/sensor_graph/graph_sensor_locations.csv')
sensor_loc

,index,sensor_id,latitude,longitude
0,0,773869,34.15497,-118.31829
1,1,767541,34.11621,-118.23799
2,2,767542,34.11641,-118.23819
3,3,717447,34.07248,-118.26772
4,4,717446,34.07142,-118.26572
...,...,...,...,...
202,202,717592,34.14604,-118.22430
203,203,717595,34.14163,-118.18290
204,204,772168,34.16542,-118.47985
205,205,718141,34.15133,-118.37456


In [71]:
ids = list(pd.read_csv('data/sensor_graph/graph_sensor_ids.txt').columns[1:6].astype(int))
ids_strings = list(pd.read_csv('data/sensor_graph/graph_sensor_ids.txt').columns[1:6])

In [72]:
sensor_loc[sensor_loc['sensor_id'].isin(ids)]

,index,sensor_id,latitude,longitude
1,1,767541,34.11621,-118.23799
2,2,767542,34.11641,-118.23819
3,3,717447,34.07248,-118.26772
4,4,717446,34.07142,-118.26572
5,5,717445,34.06913,-118.25932


In [73]:
sensor_dist = pd.read_csv('data/sensor_graph/distances_la_2012.csv')
sensor_dist

,from,to,cost
0,1201054,1201054,0.0
1,1201054,1201066,2610.9
2,1201054,1201076,2822.7
3,1201054,1201087,2911.5
4,1201054,1201100,7160.1
...,...,...,...
295369,825515,823663,9894.1
295370,825515,825494,475.6
295371,825515,825496,6458.6
295372,825515,825513,6934.2


In [74]:
sensor_dist[sensor_dist['from'].isin(ids) & sensor_dist['to'].isin(ids)]

,from,to,cost
129815,717445,717445,0.0
129816,717445,717446,2607.2
129817,717445,717447,861.6
129896,717445,767541,6596.0
129897,717445,767542,9100.7
129974,717446,717445,4464.0
129975,717446,717446,0.0
129976,717446,717447,1767.4
130057,717446,767541,6606.7
130058,717446,767542,9111.3


In [75]:
import pickle


with open('data/sensor_graph/adj_mx.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    p = u.load()

In [76]:
p[0] = list(set(p[0]).intersection(set(ids_strings)))

In [77]:
p[1] = {k : p[1][k] for k in p[0]}

In [78]:
p[2]

array([[1.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.       , 0.3909554, ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.7174379, 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]], dtype=float32)

In [79]:
p[0]

['717446', '767542', '767541', '717445', '717447']

In [80]:
p[1]

{'717446': 4, '767542': 2, '767541': 1, '717445': 5, '717447': 3}

In [82]:
indexes = list(p[1].values())
adj_new = np.zeros((len(indexes), len(indexes)))
for i in indexes:
    for j in indexes:
        val = p[2][i][j]

array([[1.        , 0.        , 0.        , 0.36143157, 0.63372165],
       [0.        , 1.        , 0.3909554 , 0.        , 0.        ],
       [0.        , 0.7174379 , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.62646437, 0.        , 0.        , 0.8948124 , 1.        ]],
      dtype=float32)

In [90]:
df = pd.read_hdf('data/metr-la.h5')

In [122]:
dfs = df[ids_strings]

In [109]:
ids[0]

767541

In [124]:
store = pd.HDFStore('data/metr-la-small.h5')
store.put('d1', dfs, format='table', data_columns=True)
store.close()

C:\Users\drobi\Anaconda3\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '767541'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
C:\Users\drobi\Anaconda3\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '767542'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
C:\Users\drobi\Anaconda3\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '717447'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribu